In [ ]:
pip install tensorflow numpy matplotlib opencv-python


In [ ]:
# This script is used to create multi-resolution TFRecords datasets with a focus on anonymizing specific features in images using pre-trained GAN models for digital media privacy enhancement.

import os
import sys
import glob
import argparse
import threading
import six.moves.queue as Queue
import traceback
import numpy as np
import tensorflow as tf
import PIL.Image
import dnnlib.tflib as tflib
from training import dataset

# Anonymization functions
def anonymize_image(img, method='pixelation'):
    """Apply anonymization methods such as pixelation or blurring."""
    if method == 'pixelation':
        factor = 10
        img = img.resize((img.width // factor, img.height // factor), PIL.Image.NEAREST)
        img = img.resize((img.width * factor, img.height * factor), PIL.Image.NEAREST)
    elif method == 'blur':
        img = img.filter(PIL.ImageFilter.GaussianBlur(radius=5))
    return img

# Dataset creation and manipulation class
class TFRecordExporter:
    def __init__(self, tfrecord_dir, expected_images, print_progress=True, progress_interval=10, anonymization_method=None):
        self.tfrecord_dir = tfrecord_dir
        self.tfr_prefix = os.path.join(self.tfrecord_dir, os.path.basename(self.tfrecord_dir))
        self.expected_images = expected_images
        self.cur_images = 0
        self.shape = None
        self.resolution_log2 = None
        self.tfr_writers = []
        self.print_progress = print_progress
        self.progress_interval = progress_interval
        self.anonymization_method = anonymization_method

        if self.print_progress:
            print('Creating anonymized dataset "%s"' % tfrecord_dir)
        if not os.path.isdir(self.tfrecord_dir):
            os.makedirs(self.tfrecord_dir)
        assert os.path.isdir(self.tfrecord_dir)

    def close(self):
        if self.print_progress:
            print('%-40s\r' % 'Flushing data...', end='', flush=True)
        for tfr_writer in self.tfr_writers:
            tfr_writer.close()
        self.tfr_writers = []
        if self.print_progress:
            print('%-40s\r' % '', end='', flush=True)
            print('Added %d anonymized images.' % self.cur_images)

    def add_image(self, img):
        if self.anonymization_method:
            img = anonymize_image(img, method=self.anonymization_method)
        if self.print_progress and self.cur_images % self.progress_interval == 0:
            print('%d / %d\r' % (self.cur_images, self.expected_images), end='', flush=True)
        if self.shape is None:
            self.shape = img.shape
            self.resolution_log2 = int(np.log2(self.shape[1]))
            assert self.shape[1] == 2**self.resolution_log2
            tfr_opt = tf.io.TFRecordOptions(compression_type=tf.io.TFRecordCompressionType.NONE)
            for lod in range(self.resolution_log2 - 1):
                tfr_file = self.tfr_prefix + '-r%02d.tfrecords' % (self.resolution_log2 - lod)
                self.tfr_writers.append(tf.io.TFRecordWriter(tfr_file, options=tfr_opt))
        assert img.shape == self.shape
        for lod, tfr_writer in enumerate(self.tfr_writers):
            if lod:
                img = img.astype(np.float32)
                img = (img[:, 0::2, 0::2] + img[:, 0::2, 1::2] + img[:, 1::2, 0::2] + img[:, 1::2, 1::2]) * 0.25
            quant = np.rint(img).clip(0, 255).astype(np.uint8)
            ex = tf.train.Example(features=tf.train.Features(feature={
                'shape': tf.train.Feature(int64_list=tf.train.Int64List(value=quant.shape)),
                'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tobytes()]))}))
            tfr_writer.write(ex.SerializeToString())
        self.cur_images += 1

# Main function
def create_anonymized_dataset(tfrecord_dir, image_dir, shuffle, method='pixelation'):
    print('Loading and anonymizing images from "%s"' % image_dir)
    image_filenames = sorted(glob.glob(os.path.join(image_dir, '*')))
    if len(image_filenames
